In [212]:
from nfl import process #saved as a package the make_feature function defined in previous notebook
import pandas as pd
import numpy as np
from sklearn.preprocessing import Imputer, RobustScaler, StandardScaler

from sklearn.linear_model import LinearRegression, ElasticNetCV

from sklearn.grid_search import GridSearchCV

from time import ctime

from sklearn.cross_validation import cross_val_score

import seaborn as sns

import matplotlib.pyplot as plt

from sklearn.cross_validation import train_test_split, ShuffleSplit
from sklearn.ensemble import ExtraTreesRegressor

import statsmodels.formula.api as smf
from statsmodels.api import add_constant

from sklearn.metrics import mean_squared_error, r2_score 

from sklearn.svm import SVR, LinearSVR

import scipy.stats as stats
import math

import time

import collections
import functools 

%matplotlib inline

In [2]:

#a memoized class taken from https://wiki.python.org/moin/PythonDecoratorLibrary#Memoize
#this caches results of a function and does not re-compute if not needed
class memoized(object):
   '''Decorator. Caches a function's return value each time it is called.
   If called later with the same arguments, the cached value is returned
   (not reevaluated).
   '''
   def __init__(self, func):
      self.func = func
      self.cache = {}
   def __call__(self, *args):
      if not isinstance(args, collections.Hashable):
         # uncacheable. a list, for instance.
         # better to not cache than blow up.
         return self.func(*args)
      if args in self.cache:
         return self.cache[args]
      else:
         value = self.func(*args)
         self.cache[args] = value
         return value
   def __repr__(self):
      '''Return the function's docstring.'''
      return self.func.__doc__
   def __get__(self, obj, objtype):
      '''Support instance methods.'''
      return functools.partial(self.__call__, obj)

In [3]:
preds = pd.read_csv("predictions_10_20.csv") #import predictions generated in notebook #3

In [6]:
preds.dropna(inplace = 1)

In [7]:
preds = preds[preds.wk < 18] #drop non reg season weeks

In [8]:
preds.head()

,Unnamed: 0,target,prediction,gid,player,pos,fname,lname,seas,wk
43,43,9.68,16.124307,3726.0,AS-1600,QB,Alex,Smith,2014,1
44,44,14.40,16.143301,3750.0,AS-1600,QB,Alex,Smith,2014,2
45,45,22.14,15.646502,3767.0,AS-1600,QB,Alex,Smith,2014,3
46,46,22.72,15.930029,3783.0,AS-1600,QB,Alex,Smith,2014,4
47,47,8.92,15.599617,3796.0,AS-1600,QB,Alex,Smith,2014,5


In [9]:
salaries = pd.read_csv("/Users/egorkharakozov/ProjectData/NFL/player_salaries.csv", header = None)

#file from rotoguru http://rotoguru1.com/cgi-bin/fyday.pl?week=1&year=2014&game=dk&scsv=1
#downloaded with importIO free scraper and converted using text to columns in excel.
# The data didnt really require a custom scraper built through python.



In [10]:
salaries.head()

,0,1,2,3,4,5,6,7,8,9,...,3932,3933,3934,3935,3936,3937,3938,3939,3940,3941
0,2014,1301,"Ryan, Matt",QB,atl,h,nor,34.42,7900 1,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1207,"Palmer, Carson",QB,ari,h,nor,28.68,6500 1,2015,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2014,1252,"Rodgers, Aaron",QB,gnb,h,nyj,32.64,7900 2,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1232,"Roethlisberger, Ben",QB,pit,h,sfo,33.66,7200 2,2015,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2014,1409,"Luck, Andrew",QB,ind,a,jac,35.30,9200 3,2014,...,2400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
#reshape and process the salary data to fit each player on a row. The data came in with each week being a row.

processed_sals = pd.DataFrame()

for i in range(0,34):
    temp = salaries.ix[i,:].reshape(438,9)
    temp = pd.DataFrame(temp)
    temp["week"] = i//2+1
    
    processed_sals = processed_sals.append(temp)


    
processed_sals.columns = ["seas", "game_id", "pname", "pos", "home_team", "h/a", "opp_team", "pts", "sal", "wk"]

processed_sals.head()

,seas,game_id,pname,pos,home_team,h/a,opp_team,pts,sal,wk
0,2014,1301,"Ryan, Matt",QB,atl,h,nor,34.42,7900 1,1
1,2014,1340,"Stafford, Matthew",QB,det,h,nyg,33.04,9000 1,1
2,2014,1409,"Luck, Andrew",QB,ind,a,den,31.7,8300 1,1
3,2014,1207,"Palmer, Carson",QB,ari,h,sdg,25.06,7500 1,1
4,2014,1263,"Cutler, Jay",QB,chi,h,buf,22.96,7800 1,1


In [12]:
processed_sals.sal = [str(_).split()[0] for _ in processed_sals.sal]

In [13]:
processed_sals.sal = processed_sals.sal.astype("float64")

In [14]:
processed_sals.dropna(inplace = 1)

In [17]:
preds["pname"] = preds.lname +", " + preds.fname

preds.head()

,Unnamed: 0,target,prediction,gid,player,pos,fname,lname,seas,wk,pname
43,43,9.68,16.124307,3726.0,AS-1600,QB,Alex,Smith,2014,1,"Smith, Alex"
44,44,14.40,16.143301,3750.0,AS-1600,QB,Alex,Smith,2014,2,"Smith, Alex"
45,45,22.14,15.646502,3767.0,AS-1600,QB,Alex,Smith,2014,3,"Smith, Alex"
46,46,22.72,15.930029,3783.0,AS-1600,QB,Alex,Smith,2014,4,"Smith, Alex"
47,47,8.92,15.599617,3796.0,AS-1600,QB,Alex,Smith,2014,5,"Smith, Alex"


In [214]:
test2 = processed_sals.merge(preds, how = "left", on = ["pname", "seas", "wk", "pos"]) 
#merge the predictions and salary data

In [215]:
test2.head()

,seas,game_id,pname,pos,home_team,h/a,opp_team,pts,sal,wk,Unnamed: 0,target,prediction,gid,player,fname,lname
0,2014,1301,"Ryan, Matt",QB,atl,h,nor,34.42,7900.0,1,1766.0,34.42,15.888593,3724.0,MR-2500,Matt,Ryan
1,2014,1340,"Stafford, Matthew",QB,det,h,nyg,33.04,9000.0,1,1951.0,33.04,16.346342,3737.0,MS-4100,Matthew,Stafford
2,2014,1409,"Luck, Andrew",QB,ind,a,den,31.7,8300.0,1,3063.0,29.70,17.701614,3736.0,AL-1100,Andrew,Luck
3,2014,1207,"Palmer, Carson",QB,ari,h,sdg,25.06,7500.0,1,672.0,27.06,16.555652,3738.0,CP-0500,Carson,Palmer
4,2014,1263,"Cutler, Jay",QB,chi,h,buf,22.96,7800.0,1,2717.0,20.96,15.572795,3725.0,JC-6200,Jay,Cutler


### In the below section I mock up the optimization script. The first function will reduce the weekly field of players, and the second function will optimize based on salary and roster limit constraints using a modified dynamic programming function similiar to the solution to the famous knapsack problem. 

In [300]:


poslist = {"QB":1, "WR":4, "RB":3, "TE":2}

def crop_weak_players(df, poslist, filtering_value = "prediction"):
    
    """returns an index on non-reduntant players by testing for 
    a dominant condition where player A is cheaper than player B and
    player A is projected for more points than player B and therefore player B is eliminated from the list.
    To avoid eliminating too many players, this function also checks the position list to see how many
    players must exist before eliminating
    parameters:
    df - a dataframe containing pos, sal, prediction, and name columns
    poslist - a dictionary for example  {"QB":1, "WR":4, "RB":3, "TE":2} showing max position limits in lineup
    """
    
    reduced_index = []
    for k, v in poslist.iteritems():
        for i1, v1 in df[df.pos==k].iterrows():
            count = 0
            for i2, v2 in df[df.pos==k].iterrows():
                if v2.name == v1.name:
                    continue
                elif (v2.sal <= v1.sal) & (v2[filtering_value] >= v1[filtering_value]):
                    count += 1
            if count >= v:
                pass
            else:
                reduced_index.append(i1)

    return reduced_index


In [217]:
#try the knapsack approach below here

In [218]:

grabcols = ["seas", "wk","pos", "pname", "sal", "target", "prediction", "player"]

test2 = test2[grabcols]

In [219]:

#here i clean up the data to trim down to a dataset per week that i will then run the knapsack optimization on

test2 = test2[(test2.wk == 16) & (test2.seas == 2015)]
tot_sal = 500 -  np.round(np.mean(test2.sal[test2.pos == "Def"]),-2)/100
test2 = test2[test2.pos != "Def"]
print test2.pos.value_counts()

test2.sal[test2.sal < 2000] = 0 #2k is min salary so anything under that is a typo or bad data point
test2.sal.replace(0, int(np.round(np.mean(test2.sal),-2)), inplace = 1)

test2.sal = [int(_)/100 for _ in test2.sal]
test2.dropna(inplace = 1)
test2.prediction = [int(np.rint(_*100)) for _ in test2.prediction]
#test2 = test2.sort_values(by = ["prediction"], ascending=False)

print tot_sal
test2.describe()

WR    146
RB    119
TE     91
QB     39
Name: pos, dtype: int64
473.0


,wk,sal,target,prediction
count,331.0,331.000000,331.000000,331.000000
mean,16.0,39.401813,8.380242,875.302115
std,0.0,13.925512,8.181730,404.846682
min,16.0,25.000000,-1.000000,298.000000
25%,16.0,30.000000,1.800000,559.000000
50%,16.0,33.000000,6.300000,762.000000
75%,16.0,49.000000,13.000000,1095.000000
max,16.0,93.000000,36.900000,2108.000000


In [220]:
reduced_index = crop_weak_players(test2, poslist)
test2 = test2.ix[reduced_index]

In [221]:

test2.reset_index(drop = 1, inplace= 1)

#test2

In [222]:
#add weights for positions as that will be used as a weight
test2["pos2"] = test2["pos"]
test2 = pd.get_dummies(test2, columns = ["pos2"])

In [223]:

#add a weight for flex (rb + wr)
test2["tot"] = np.sum(test2.ix[:,-4:], axis = 1)
test2["rbwr"] = np.sum(test2.ix[:,["pos2_RB", "pos2_WR"]], axis = 1)
test2["rbte"] = np.sum(test2.ix[:,["pos2_RB", "pos2_TE"]], axis = 1)
test2["wrte"] = np.sum(test2.ix[:,["pos2_WR", "pos2_TE"]], axis = 1)
test2.head()

,seas,wk,pos,pname,sal,target,prediction,player,pos2_QB,pos2_RB,pos2_TE,pos2_WR,tot,rbwr,rbte,wrte
0,2015,16,QB,"Cousins, Kirk",55,35.20,1872,KC-2350,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,2015,16,QB,"Fitzpatrick, Ryan",52,24.14,1870,RF-0500,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,2015,16,QB,"Tannehill, Ryan",51,18.96,1687,RT-0150,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,2015,16,QB,"Gabbert, Blaine",50,16.90,1526,BG-0100,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,2015,16,QB,"Manning, Eli",56,7.36,1981,EM-0200,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [224]:
#convert to integers as that will create discrete values and take advantage of the memoized construct

for col in ["pos2_QB", "pos2_RB", "pos2_WR", "pos2_TE", "tot", "rbwr", "rbte", "wrte"]:
    test2.ix[:,col] = test2.ix[:,col].astype("int")


In [230]:
def knapsack2(items, maxweight, poslist, rbwr = 6, rbte = 4, wrte = 5, roster = 8, value = "prediction"):
    """
    Solve the knapsack problem by finding the most valuable
    subsequence of `items` subject that weighs no more than
    the weight constraints.

    `items` is a df returned by the filter_weak_players with added columns for constraints.

    `maxweight` is the salary limit
    
    `poslist` is a dictionary with positional max size limits
    
    `flex` is the max total rb + wr allowed
    
    `roster` is total roster size

    This returns a dataframe with the optimal lineup selected and the total predicted score
    """
    
    grabcols = ["pname", "pos", "sal", "pos2_QB", "pos2_WR", "pos2_RB","pos2_TE", "rbwr", "rbte", "wrte", "tot", value]
    items = items[grabcols]
    # Return the value of the most valuable subsequence of the first i
    # elements in items whose weights sum to no more than j.
    @memoized
    def bestvalue(i, j, k, l, m, n, o, p, q, r):
        if i == 0: return 0
        _, pos, weight, qbw, wrw, rbw, tew, fx1, fx2, fx3, tw, val = items.ix[i - 1, :]
        if ((weight > j) | (qbw > k) | (wrw > l) | (rbw > m) | (tew > n )\
            | (fx1 > o) |(fx2 > p) |(fx3 > q) | (tw > p)) :
            return bestvalue(i - 1, j, k, l, m, n, o, p, q, r)
        else:
            return max(bestvalue(i - 1, j, k, l, m, n, o, p, q, r),
                       bestvalue(i - 1, j - weight, k-qbw, l-wrw, m-rbw, n-tew,\
                                 o-fx1, p - fx2, q- fx3, r - tw) + val)

    j = maxweight
    k = poslist["QB"]
    l = poslist["WR"]
    m = poslist["RB"]
    n = poslist["TE"]
    o = rbwr
    p = rbte
    q = wrte
    r = roster 
    ### total roster of 9 constraint, would have to be manually changed if optimizing for other websites
    
    
    result = pd.DataFrame(columns = ["pname", "pos", "sal", "pos2_QB", "pos2_WR",
                                     "pos2_RB","pos2_TE", "rbwr", "rbte", "wrte", "tot", value])
    for i in xrange(len(items), 0, -1):
        if bestvalue(i, j, k, l, m, n, o, p, q, r) != bestvalue(i - 1, j, k, l, m, n, o, p, q, r):
            result =result.append(items.ix[i - 1, :])
            j -= items.ix[i - 1, :].sal
            k -= items.ix[i - 1, :].pos2_QB
            l -= items.ix[i - 1, :].pos2_WR
            m -= items.ix[i - 1, :].pos2_RB
            n -= items.ix[i - 1, :].pos2_TE
            o -= items.ix[i - 1, :].rbwr
            p -= items.ix[i - 1, :].rbte
            q -= items.ix[i - 1, :].wrte
            r -= items.ix[i - 1, :].tot
           
    return bestvalue(len(items), maxweight, poslist["QB"], 
                     poslist["WR"], poslist["RB"], poslist["TE"], rbwr, rbte, wrte, roster)/100.0, result

In [348]:
a = time.time()
score2 , lineup2 = knapsack2(test2, tot_sal, poslist)

print time.time() - a

66.0968039036


In [349]:
lineup2

,pname,pos,sal,pos2_QB,pos2_WR,pos2_RB,pos2_TE,rbwr,rbte,wrte,tot,prediction
54,"Reed, Jordan",TE,65.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1476.0
38,"Johnson, David",RB,58.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1650.0
37,"Gore, Frank",RB,40.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1118.0
25,"Boldin, Anquan",WR,37.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1278.0
18,"Brown, Antonio",WR,93.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1937.0
14,"Maclin, Jeremy",WR,60.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1538.0
10,"Landry, Jarvis",WR,64.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1474.0
4,"Manning, Eli",QB,56.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1981.0


In [350]:
score2

124.52

In [351]:
sum(lineup2.prediction/100.0)

124.52

In [235]:
### as a quick sanity test, setting a trivially large salary constraint 
#should return just the maximum predicted lineup.
#which it does

score2 , lineup3 = knapsack2(test2, 99999999, poslist)
lineup3

,pname,pos,sal,pos2_QB,pos2_WR,pos2_RB,pos2_TE,rbwr,rbte,wrte,tot,prediction
54,"Reed, Jordan",TE,65.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1476.0
41,"Freeman, Devonta",RB,74.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1510.0
38,"Johnson, David",RB,58.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1650.0
18,"Brown, Antonio",WR,93.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1937.0
14,"Maclin, Jeremy",WR,60.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1538.0
7,"Marshall, Brandon",WR,76.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1570.0
6,"Jones, Julio",WR,83.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1756.0
5,"Roethlisberger, Ben",QB,68.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2108.0


In [234]:
sum(lineup2.sal) +500 - tot_sal

500.0

### Now that I have a function that trims each weekly player set and generates the optimal lineup, I can create a loop to generate the optimized line ups per week and see how they perform against scores likely that would have needed to cash. The best source I have found so far on average scores needed is here 
https://rotogrinders.com/articles/what-s-needed-to-win-on-draftkings-368296
### and this article stipulates that although the average score is 129, because the distribution has fatter tails, (platykurtic) in stats speak, the last winning score is closer to the 119 mark. Given that this article is from 2014 we can split the difference and test the 34 predicted weeks against both levels and the midpoint to gauge the model's performance.


In [272]:
#first I create a df with all of the predicted points and salaries in one frame:

all_data = processed_sals.merge(preds, how = "left", on = ["pname", "seas", "wk", "pos"])

In [273]:
all_data.head()

,seas,game_id,pname,pos,home_team,h/a,opp_team,pts,sal,wk,Unnamed: 0,target,prediction,gid,player,fname,lname
0,2014,1301,"Ryan, Matt",QB,atl,h,nor,34.42,7900.0,1,1766.0,34.42,15.888593,3724.0,MR-2500,Matt,Ryan
1,2014,1340,"Stafford, Matthew",QB,det,h,nyg,33.04,9000.0,1,1951.0,33.04,16.346342,3737.0,MS-4100,Matthew,Stafford
2,2014,1409,"Luck, Andrew",QB,ind,a,den,31.7,8300.0,1,3063.0,29.70,17.701614,3736.0,AL-1100,Andrew,Luck
3,2014,1207,"Palmer, Carson",QB,ari,h,sdg,25.06,7500.0,1,672.0,27.06,16.555652,3738.0,CP-0500,Carson,Palmer
4,2014,1263,"Cutler, Jay",QB,chi,h,buf,22.96,7800.0,1,2717.0,20.96,15.572795,3725.0,JC-6200,Jay,Cutler


In [274]:
grabcols = ["seas", "wk","pos", "pname", "sal", "target", "prediction", "player","pts"]

poslist = {"QB":1, "WR":4, "RB":3, "TE":2}


all_data = all_data[grabcols]

### PART ONE GENERATE THE OPTIMAL LINE UP FOR EACH WEEK USING THE PREDICTION MODEL RESULTS

optimized_lineups = pd.DataFrame()

for year in [2014, 2015]:
    for week in set(all_data.wk):
        mask = ((all_data.seas == year) & (all_data.wk == week))
        temp = all_data[mask]
        def_sal = np.round(np.mean(temp.sal[temp.pos == "Def"]),-2)/100
        def_pts = np.round(np.mean(temp.pts[temp.pos == "Def"]),-2)/100
        tot_sal = 500 - def_sal
        temp = temp[temp.pos != "Def"]
        temp.sal[temp.sal <2000] = 0
        temp.sal.replace(0, int(np.round(np.mean(temp.sal),-2)), inplace = 1)
        temp.sal = [int(_)/100 for _ in temp.sal]
        temp.dropna(inplace = 1)
        temp.prediction = [int(np.rint(_*100)) for _ in temp.prediction]
        temp.target = [int(np.rint(_*100)) for _ in temp.target]
        
        reduced_index = crop_weak_players(temp, poslist)
        
        
        temp = temp.ix[reduced_index]
        temp.reset_index(drop =1 , inplace = 1)
        temp["pos2"] = temp["pos"]
        temp = pd.get_dummies(temp, columns = ["pos2"])
        temp["tot"] = np.sum(temp.ix[:,-4:], axis = 1)
        
        
        temp["rbwr"] = np.sum(temp.ix[:,["pos2_RB", "pos2_WR"]], axis = 1)
        temp["rbte"] = np.sum(temp.ix[:,["pos2_RB", "pos2_TE"]], axis = 1)
        temp["wrte"] = np.sum(temp.ix[:,["pos2_WR", "pos2_TE"]], axis = 1)
                #convert to integers as that will create discrete values and take advantage of the memoized construct
        for col in ["pos2_QB", "pos2_RB", "pos2_WR", "pos2_TE", "tot", "rbwr", "rbte", "wrte"]:
            temp.ix[:,col] = temp.ix[:,col].astype("int")
            
        _ , lineup = knapsack2(temp, tot_sal, poslist)
        
        
        lineup["seas"] = year
        lineup["wk"] = week
        lineup["def_sal"] = def_sal/8 
        lineup["def_pts"] = def_pts/8.0
        #add one eights of the def_sal to each week so that when i sum i can double check each week is under limit
        
        optimized_lineups = optimized_lineups.append(lineup)
        print "Done with year {} week {}".format(year, week)
        
        
        

Done with year 2014 week 1
Done with year 2014 week 2
Done with year 2014 week 3
Done with year 2014 week 4
Done with year 2014 week 5
Done with year 2014 week 6
Done with year 2014 week 7
Done with year 2014 week 8
Done with year 2014 week 9
Done with year 2014 week 10
Done with year 2014 week 11
Done with year 2014 week 12
Done with year 2014 week 13
Done with year 2014 week 14
Done with year 2014 week 15
Done with year 2014 week 16
Done with year 2014 week 17
Done with year 2015 week 1
Done with year 2015 week 2
Done with year 2015 week 3
Done with year 2015 week 4
Done with year 2015 week 5
Done with year 2015 week 6
Done with year 2015 week 7
Done with year 2015 week 8
Done with year 2015 week 9
Done with year 2015 week 10
Done with year 2015 week 11
Done with year 2015 week 12
Done with year 2015 week 13
Done with year 2015 week 14
Done with year 2015 week 15
Done with year 2015 week 16
Done with year 2015 week 17


In [275]:
#merge with preds to get the actual scores in
optimized_lineups = optimized_lineups.merge(preds[["pname", "seas", "wk", "target"]], on= ["pname", "seas", "wk"])

optimized_lineups["prediction"] = optimized_lineups["prediction"] / 100.0

In [282]:
optimized_lineups["tot_sal"] = optimized_lineups.sal + optimized_lineups.def_sal
optimized_lineups["tot_score"] = optimized_lineups.def_pts + optimized_lineups.target

In [284]:
#optimized_lineups.drop(258,inplace = 1) 
#there are two RBs named david johnshon on this week so when i merge to bring in the target score i get a dupe


In [278]:
optimized_lineups[(optimized_lineups.wk == 16) & (optimized_lineups.seas == 2015)]

,pname,pos,sal,pos2_QB,pos2_WR,pos2_RB,pos2_TE,rbwr,rbte,wrte,tot,prediction,seas,wk,def_sal,def_pts,target,tot_sal
256,"Reed, Jordan",TE,65.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,14.76,2015,16,3.375,0.0,36.90,68.375
257,"Johnson, David",RB,58.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,16.50,2015,16,3.375,0.0,21.70,61.375
259,"Gore, Frank",RB,40.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,11.18,2015,16,3.375,0.0,23.50,43.375
260,"Boldin, Anquan",WR,37.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,12.78,2015,16,3.375,0.0,7.70,40.375
261,"Brown, Antonio",WR,93.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,19.37,2015,16,3.375,0.0,13.80,96.375
262,"Maclin, Jeremy",WR,60.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,15.38,2015,16,3.375,0.0,15.90,63.375
263,"Landry, Jarvis",WR,64.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,14.74,2015,16,3.375,0.0,21.10,67.375
264,"Manning, Eli",QB,56.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,19.81,2015,16,3.375,0.0,7.36,59.375


In [286]:
a = optimized_lineups.pivot_table(values = ["prediction", "tot_score", "tot_sal"], index = ["seas", "wk"], aggfunc = "sum")

In [301]:
len(a[a.tot_score>124]) #number of weeks that would have won given the 119, 124, 129 point thresholds
# I derive them from this article and take the midpoint between mean 129 and projected 119. 
#https://rotogrinders.com/articles/what-s-needed-to-win-on-draftkings-368296
    

19

In [302]:
len(a[a.tot_score > 119])

21

In [303]:
len(a[a.tot_score > 129])

17

In [359]:
-(50*34) + 90*21  #amount of money won over 2 years playing a $50 50/50 once a week 
#(of course assuming no reinvestment and flat hurdle of 119 to win and also an average defense)

190